# manually check all segmentationmap

In [1]:
import  numpy               as     np
from    scripts.tools       import *
from    scripts.masking     import *
from    astropy.table       import Table
from    astropy.io          import fits
from    tqdm                import tqdm
import  os
import  matplotlib.pyplot   as     plt
import  time
%matplotlib auto


obj_lis = Table.read('obj_lis_selected.fits')
for obj in tqdm(obj_lis):
    if obj['manual_select'] == 'selected': continue
    path = f"data_extracted/{file_name(obj,prefix='extracted')}"
    with fits.open(path,mode='update') as hdu:
        seg     = find_data('SEG',hdu)[1]
        if find_data('SEG_MOD',hdu) != None: seg = find_data('SEG_MOD',hdu)[1]
        seg.data = np.where(seg.data == obj['ID'],obj['ID'],0)
        line_maps = [find_data('DSCI',hdu)[1],
                     find_data('LINE_HA',hdu)[1],
                     find_data('LINE_HB',hdu)[1]]
        seg_var = run_segmentation_editor(line_maps,seg,obj['ID'])
        seg_var.name = 'SEG_MOD'
        fits.ImageHDU(data=seg_var.data,header=seg_var.header).writeto(f"seg/{file_name(obj,'seg')}",overwrite=True)
        hdu = save_update(seg_var,hdu)
        hdu.flush()

Using matplotlib backend: <object object at 0x000001B83DFD56E0>


  0%|          | 0/158 [00:00<?, ?it/s]Traceback (most recent call last):
  File "c:\Users\26715\.conda\envs\seg\lib\site-packages\matplotlib\cbook.py", line 298, in process
    func(*args, **kwargs)
  File "l:\Balmer-Decrement_3DHST\scripts\masking.py", line 172, in on_click
    self.modify_segmentation(x, x + 1, y, y + 1, add_only=True)
  File "l:\Balmer-Decrement_3DHST\scripts\masking.py", line 161, in modify_segmentation
    self.update_images()
  File "l:\Balmer-Decrement_3DHST\scripts\masking.py", line 137, in update_images
    rgba_img[seg_mask, :3] = plt.cm.plasma_r(norm_inside(img[seg_mask]))[:, :3]  # Apply inside norm
  File "c:\Users\26715\.conda\envs\seg\lib\site-packages\matplotlib\colors.py", line 1779, in __call__
    raise ValueError("vmin must be less or equal to vmax")
ValueError: vmin must be less or equal to vmax
Traceback (most recent call last):
  File "c:\Users\26715\.conda\envs\seg\lib\site-packages\matplotlib\cbook.py", line 298, in process
    func(*args, **k

KeyboardInterrupt: 

#save segs

In [3]:
from    astropy.table       import Table
from    astropy.io          import fits
from    tqdm                import tqdm
from    scripts.tools       import *
obj_lis = Table.read('obj_lis_selected.fits')
for obj in tqdm(obj_lis):
    try:
        path = f"data_extracted/{file_name(obj,prefix='extracted')}"
        path_seg = f"seg/{file_name(obj,'seg')}"
        with fits.open(path,mode='update') as hdu:
            with fits.open(path_seg) as seg:
                seg[1].name = 'SEG_MOD'
                to_save = fits.ImageHDU(data = seg[1].data,name='SEG_MOD')
                save_update(to_save,hdu)
                hdu.flush()
    except Exception as e:
            print(f"! {obj['subfield']}-{obj['ID']} failed, error{e}")

100%|██████████| 158/158 [01:15<00:00,  2.09it/s]
